In [8]:
from odata_client import ODataClient
from mcp_server import MCPServer
from dotenv import load_dotenv
import os
from typing import Annotated, Any, Dict, List, Optional, Union, Tuple
from pydantic import Field
import asyncio


load_dotenv()


True

In [9]:
def _json_ready(x: Any) -> Any:
    """Convert dataclasses/pydantic/objects to JSON-serialisable structures."""
    if x is None or isinstance(x, (str, int, float, bool)):
        return x
    if isinstance(x, (list, tuple)):
        return [_json_ready(i) for i in x]
    if isinstance(x, dict):
        return {k: _json_ready(v) for k, v in x.items()}
    if hasattr(x, "model_dump"):
        try:
            return _json_ready(x.model_dump())
        except Exception:
            pass
    try:
        from dataclasses import is_dataclass, asdict

        if is_dataclass(x):
            return _json_ready(asdict(x))
    except Exception:
        pass
    if hasattr(x, "dict") and callable(getattr(x, "dict")):
        try:
            return _json_ready(x.dict())
        except Exception:
            pass
    if hasattr(x, "__dict__"):
        try:
            return _json_ready(vars(x))
        except Exception:
            pass
    return repr(x)

In [10]:
BASE_URL = os.getenv("MCP_1C_BASE", "")
USERNAME = os.getenv("ONEC_USERNAME")
PASSWORD = os.getenv("ONEC_PASSWORD")
VERIFY_SSL = os.getenv("ONEC_VERIFY_SSL", "false").lower() not in {"false", "0", "no"}

print('Creds:')
print(BASE_URL)
print(USERNAME)
print(PASSWORD)
print(VERIFY_SSL)

_server = MCPServer(base_url=BASE_URL, username=USERNAME, password=PASSWORD, verify_ssl=VERIFY_SSL)

Creds:
http://192.168.18.113/TEST19/odata/standard.odata
nikita
nikita2000
False


2025-08-22 00:06:32,235 [INFO] mcp_server: 1C OData metadata cache warmed up successfully


In [12]:
async def resolve_field_name(
    object_name: Annotated[str, Field(
        description="Точное имя набора сущностей (EntitySet) из OData сервиса 1С",
        examples=["Catalog_Контрагенты", "Document_ПлатежноеПоручение", "Catalog_Номенклатура"],
        max_length=256
    )],
    user_field: Annotated[str, Field(
        description="Название поля на русском языке или общепринятое обозначение",
        examples=["Наименование", "ИНН", "Дата", "Код", "Сумма", "Номер"],
        max_length=256
    )]
) -> Dict[str, Any]:
    """
    Нормализует человеческое название поля в точное техническое имя свойства OData.
    Учитывает синонимы и общепринятые названия полей в системе 1С.
    Функция использует заранее загруженные метаданные OData,
    поэтому требует предварительного вызова mcp_tool_metadata() или get_schema()
    
    Args:
      object_name: Точное имя набора сущностей (EntitySet), полученное из resolve_entity_name()
      user_field: Название поля на русском языке или общепринятое обозначение
                 (например: "Название", "номер", "ИНН", "Дата", "Код")
    
    Returns:
      Dict с следующими полями:
        - resolved (str|null): Найденное точное имя свойства в формате OData
                        (например: "Description", "ИНН", "Date", "Code")
                        или None, если совпадение не найдено
        - http_code (int|null): HTTP статус код запроса метаданных
        - http_message (str|null): Текстовое описание HTTP статуса
        - odata_error_code (str|null): Код ошибки OData, если произошла ошибка
        - odata_error_message (str|null): Детальное сообщение об ошибке OData
    
    Примеры использования:
        - Получить название поля, содержащего название контрагента: resolve_field_name("Catalog_Контрагенты", "название") → {"resolved": "Description"}
        - Узнать, в каком поле сущности платежного поручения хранится дата создания платежного поручения: resolve_field_name("Document_ПлатежноеПоручение", "Дата") → {"resolved": "Date"}
    """
    resolved = await asyncio.to_thread(_server.resolve_field_name, object_name, user_field)
    return _json_ready(
        {
            "resolved": resolved,
            "http_code": _server.client.get_http_code(),
            "http_message": _server.client.get_http_message(),
            "odata_error_code": _server.client.get_error_code(),
            "odata_error_message": _server.client.get_error_message(),
        }
    )

In [18]:
res = await resolve_field_name('Document_ПлатежноеПоручение', 'Контрагент_Key')
res

{'resolved': 'Контрагент',
 'http_code': 200,
 'http_message': 'OK',
 'odata_error_code': None,
 'odata_error_message': None}

In [ ]:
async def search_object(
    user_type: Annotated[str, Field(
        description="Тип сущности для поиска",
        examples=["справочник", "документ", "регистр", "константа"],
        max_length=256
    )],
    user_entity: Annotated[str, Field(
        description="Название сущности на русском языке",
        examples=["Контрагенты", "Платежное поручение", "Номенклатура", "Валюты"],
        max_length=256
    )],
    user_filters: Annotated[Optional[Union[str, Dict[str, Any], List[str]]], Field(
        description="Условия поиска. Допустимые варианты:" \
                   "1) Строка - текст для поиска по основному полю. " \
                   "Если содержит ключевые слова OData (eq, ge, и т.д.), будет " \
                   "использована как готовый `$filter`." \
                   "2) Словарь поле: значение. Строковые значения ищутся " \
                   "прогрессивно, числа и даты передаются как есть." \
                   "3) Список - набор готовых выражений, объединяемых через AND." \
                   "4) None - возврат первых записей без фильтра",
        examples=[
            "Number eq 'БП-000777'",
            {"Номер": "123", "СуммаДокумента": 1000},
            ["Date eq datetime'2024-01-19T00:00:00'", "СуммаДокумента ge 1000", "Number eq 'ККП-000727'"],
        ]
    )] = None,
    top: Annotated[int, Field(
        description="Ограничение количества возвращаемых записей",
        examples=[1, 5, 10],
        ge=1
    )] = 1,
    expand: Annotated[str, Field(
        description="Поля для расширения связанных данных (через запятую)",
        examples=["Контрагент", "Склад,Номенклатура", "Владелец"]
    )] = None
) -> Dict[str, Any]:
    """
    Выполняет интеллектуальный поиск по сущности с автоматическим разрешением имен
    и прогрессивной стратегией поиска для повышения вероятности нахождения результатов.
    Функция автоматически определяет оптимальные поля для текстового поиска и
    последовательно применяет стратегии: точное совпадение → частичное совпадение →
    регистронезависимый поиск → исключение папок (если применимо).
    Если фильтр передан готовым выражением OData, он используется напрямую.
    Имена полей нормализуются автоматически.
    
    Args:
      user_type: Тип сущности для уточнения поиска (например: "справочник", "документ")
      user_entity: Название сущности на русском языке
      user_filters: Условия поиска. Может быть:
                   - Строка: текст либо готовое OData-выражение
                   - Словарь: {поле: значение} с автоматической подстановкой типов
                   - Список: выражения для объединения через AND
                   - None: возврат первых записей без фильтра
      top: Максимальное количество возвращаемых записей
      expand: Поля для включения связанных данных
    
    Returns:
      Dict с следующими полями:
        - http_code (int|null): HTTP статус код запроса
        - http_message (str|null): Текстовое описание HTTP статуса
        - odata_error_code (str|null): Код ошибки OData, если произошла ошибка
        - odata_error_message (str|null): Детальное сообщение об ошибке OData
        - data (dict|list|null): Найденные данные. При top=1 возвращает один объект, 
                                при top>1 возвращает список объектов
    
    Примеры использования:
      - Найти контрагента по наименованию:
        search_object("справочник", "Контрагенты", "ООО Ромашка")
      - Найти документ по номеру:
        search_object("документ", "Платежное поручение", {"Number": "00003"})
      - Найти документы по дате и сумме:
        search_object(
            "документ", "Платежное поручение",
            ["Date eq datetime'2024-01-19T00:00:00'", "СуммаДокумента ge 1000"],
            top=10
        )
      - Найти несколько номенклатур:
        search_object("справочник", "Номенклатура", "товар", top=5)
    """
    res = await asyncio.to_thread(_server.search_object, user_type, user_entity, user_filters, top, expand)
    return _json_ready(res)

In [55]:
# res = await search_object("документ", "Платежное поручение", ["Number eq 'ККП-727777'"], 5)
# res = await search_object("справочник", "Контрагенты", {"Description": "ООО Рог и Копыто"}, 5)
# res = await search_object("справочник", "Контрагенты", {"Code": "РК-00001"}, 5)
# res = await search_object("документ", "Платежное поручение", {"Number": "УУГ-000001"}, 1)
res = await search_object("документ", "Платежное поручение", "Date eq datetime'2025-08-21T00:00:00'", top=1)
# res = await search_object("документ", "Платежное поручение", {"Date": "datetime'2025-08-21T00:00:00'"}, top=1)
res

2025-08-22 00:55:08,751 [INFO] mcp_server: search_object: type=документ entity=Платежное поручение filters_type=str top=1 expand=None filters="Date eq datetime'2025-08-21T00:00:00'"


{'http_code': 200,
 'http_message': 'OK',
 'odata_error_code': None,
 'odata_error_message': None,
 'last_id': None,
 'data': {'Ref_Key': '922f348b-7e69-11f0-a365-02fff02495ff',
  'DataVersion': 'AAAAAAAdbS4=',
  'DeletionMark': False,
  'Number': 'УКП-000001',
  'Date': '2025-08-21T00:00:00',
  'Posted': False,
  'ДокументОснование': '',
  'ДокументОснование_Type': 'StandardODATA.Undefined',
  'Организация_Key': '00000000-0000-0000-0000-000000000000',
  'Налог_Key': '00000000-0000-0000-0000-000000000000',
  'ВидНалоговогоОбязательства': 'Налог',
  'СчетОрганизации_Key': '00000000-0000-0000-0000-000000000000',
  'Контрагент': '328578d5-7812-11f0-a365-02fff02495ff',
  'Контрагент_Type': 'StandardODATA.Catalog_Контрагенты',
  'СчетКонтрагента_Key': '00000000-0000-0000-0000-000000000000',
  'СуммаДокумента': 7579.79,
  'СтавкаНДС': 'НДС20',
  'СуммаНДС': 1263.3,
  'ВидПлатежа': '',
  'ОчередностьПлатежа': 5,
  'НазначениеПлатежа': 'Предоплата за электроэлементы по счету-оферте 971686Z от 

In [56]:
async def find_object(
    object_name: Annotated[str, Field(
        description="Точное имя набора сущностей (EntitySet) из OData сервиса 1С",
        examples=["Catalog_Контрагенты", "Document_ПлатежноеПоручение", "Catalog_Номенклатура"],
        max_length=256
    )],
    filters: Annotated[Union[str, Dict[str, Any], List[str]], Field(
        description="Условия фильтрации для поиска записи. Может быть: "
                   "1) Строка с выражением $filter (например: \"Code eq 'БП-000001'\") "
                   "2) Словарь поле: значение (строковые значения автоматически экранируются) "
                   "3) Список выражений, которые объединяются через AND",
        examples=[
            "Code eq 'БП-000001'",
            {"Code": "БП-000001", "DeletionMark": False},
            ["Code eq 'БП-000001'", "Description ne ''"]
        ]
    )] = None,
    expand: Annotated[str, Field(
        description="Поля для расширения связанных данных (OData $expand). "
                   "Указываются через запятую",
        examples=["Контрагент", "Склад,Номенклатура", "Владелец"]
    )] = None
) -> Dict[str, Any]:
    """
    Находит первую запись в указанном наборе сущностей, соответствующую условиям фильтрации.
    Эквивалентно вызову list_objects(..., top=1) с возвратом одной записи вместо списка.
    Функция полезна для поиска конкретного объекта по уникальным характеристикам 
    (коду, наименованию, идентификатору) когда ожидается только один результат.
    Если требуется получить несколько записей, следует использовать функцию list_objects вместо find_object.
    
    Args:
      object_name: Точное имя набора сущностей (EntitySet), полученное из resolve_entity_name()
      filters: Условия фильтрации для поиска записи. Поддерживает различные форматы:
              - Строка с выражением OData $filter
              - Словарь {имя_поля: значение} для простых равенств
              - Список выражений для объединения через AND
      expand: Поля для включения связанных данных (разделитель - запятая)
    
    Returns:
      Dict с следующими полями:
        - http_code (int|null): HTTP статус код запроса
        - http_message (str|null): Текстовое описание HTTP статуса
        - odata_error_code (str|null): Код ошибки OData, если произошла ошибка
        - odata_error_message (str|null): Детальное сообщение об ошибке OData
        - data (dict|null): Найденная запись в виде словаря или None, если ничего не найдено

    Примеры использования:
        - Найти контрагента с кодом "БП-000001": 
          find_object("Catalog_Контрагенты", filters={"Code": "БП-000001"})
        - Найти незаполненное платежное поручение: 
          find_object("Document_ПлатежноеПоручение", filters={"Posted": False})
        - Найти номенклатуру с расширением данных единицы измерения:
          find_object("Catalog_Номенклатура", filters={"Description": "Техническая поддержка ПО"}, expand="ЕдиницаИзмерения")
    """
    data = await asyncio.to_thread(_server.find_object, object_name, filters, expand)
    return _json_ready(data)

In [74]:
res = await find_object("Catalog_Контрагенты", {"Description": "ООО Рог и Копыто"})
expanded_res = await find_object("Catalog_Контрагенты_КонтактнаяИнформация", {"Ref_Key": res["data"]["Ref_Key"]}
)
expanded_res

{'http_code': 200,
 'http_message': 'OK',
 'odata_error_code': None,
 'odata_error_message': None,
 'last_id': None,
 'data': None}